In [1]:
pip install -qU langchain langchain-openai langchain-anthropic langchain-community wikipedia langchain-pinecone

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
import os 
import getpass

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

llm = ChatOpenAI(model = "gpt-3.5-turbo")
os.environ['USER_AGENT'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
os.environ["PINECONE_API_KEY"] = getpass.getpass()

········


In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = OpenAIEmbeddings()
vectorstore = PineconeVectorStore(index_name="test-example", embedding=embeddings)

vectorstore.add_texts(["i am working on langchain-basics"], namespace="langchain")
vectorstore.add_texts(["i am working on data structures and algorithms"], namespace="dsa")


/home/jovyan/.local/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


['b62f401f-9ac3-42ed-927f-88391c89aaa2']

In [3]:
vectorstore.as_retriever(search_kwargs={"namespace": "dsa"}).get_relevant_documents(
    "what am i workikng on?"
)

/home/jovyan/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='i am working on data structures and algorithms'),
 Document(page_content='i am working on data structures and algorithms')]

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    ConfigurableField,
    RunnablePassthrough,
)

template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retriever = vectorstore.as_retriever()

In [11]:
configurable_retriever = retriever.configurable_fields(
    search_kwargs = ConfigurableField(
        id = "search_kwargs",
        name = "Search Kwargs",
        description = "The search kwargs to use",
    )
)

In [12]:
chain = (
    {"context" : configurable_retriever,"question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
chain.invoke(
    "what am I working on?",
    config = {"configurable" : {"search_kwargs" : {"namespace" : "dsa"}}},
)

'You are working on data structures and algorithms.'

In [15]:
chain.invoke(
    "where did the user work?",
    config={"configurable": {"search_kwargs": {"namespace": "ankush"}}},
)

'It is not possible to determine where the user worked based on the given context as there is no information provided.'